<a href="https://colab.research.google.com/github/tmdoi/small-Japanese-LLM-compare/blob/main/tokyotech_llm_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

情報源
[
tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5](https://huggingface.co/tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5)

In [ ]:
!nvidia-smi

In [ ]:
%pip -q install --upgrade pip
%pip -q install "vllm-cu121>=0.5.0" transformers accelerate huggingface_hub hf_transfer
print("インストール完了：このセルの後、【ランタイムを再起動】してください。")

In [ ]:
import os

# vLLM が要求するマルチプロセス方式を事前に明示
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

# 単一GPUを明示（Colab L4想定）
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# まれな相性問題の回避に有効な場合あり（必要時のみ有効化）
# os.environ["VLLM_USE_TRITON_FLASH_ATTENTION"] = "0"

print("環境変数を設定しました。")


In [ ]:
from huggingface_hub import login
login()  # 対話プロンプトに従って hf_ から始まるトークンを貼り付け


In [ ]:
from vllm import LLM, SamplingParams

MODEL_ID = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5"

llm = LLM(
    model=MODEL_ID,
    dtype="auto",            # L4 なら自動で bfloat16 を選択
    max_model_len=4096,      # 必要に応じて後で 8192 に拡大（VRAMと相談）
    trust_remote_code=True,
)

params = SamplingParams(
    temperature=0.6,
    top_p=0.9,
    max_tokens=128,
)

messages = [
    {"role": "system", "content": "あなたは礼儀正しく明晰な日本語で応答するアシスタントです。"},
    {"role": "user", "content": "短く自己紹介してください。"}
]

out = llm.chat(messages=messages, sampling_params=params)
print(out[0].outputs[0].text)


In [ ]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5"

tokenizer = AutoTokenizer.from_pretrained(model_name)

llm = LLM(
    model=model_name,
    dtype="auto",
    max_model_len=4096,
    trust_remote_code=True,
    tensor_parallel_size=1,
)

sampling_params = SamplingParams(
    temperature=0.6,
    top_p=0.9,
    max_tokens=512,
    # stop はまず未指定（特殊トークン名の不一致で止まらないケースを避ける）
)

messages = [
    {
        "role": "user",
        "content": "東京の紅葉した公園で、東京タワーと高層ビルを背景に、空を舞うツバメと草地に佇むラマが出会う温かな物語を書いてください。",
    },
]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

outputs = llm.generate([prompt], sampling_params)  # ★ リストで渡す
print(outputs[0].outputs[0].text)
